In [3]:
from bs4 import BeautifulSoup
import requests
import MeCab as mc
import json
import sqlite3
import requests
import records
import glob
import json
import urllib.parse
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import gensim
from gensim.models import Word2Vec
from gensim.corpora.dictionary import Dictionary
from gensim import corpora, matutils
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV


In [5]:
conn = None

def connect():
    global conn
    conn = sqlite3.connect('./yahoo_1.db')
    
def close():
    conn.close()
    
def create_table():
    conn.execute('DROP TABLE IF EXISTS NEWS')
    conn.execute('''CREATE TABLE NEWS(
    id               INTEGER PRIMARY KEY AUTOINCREMENT,
    title            TEXT,
    category         TEXT
    
    )''')
    
def load(title,category):
    conn.execute(
    'INSERT INTO news (title,category) VALUES (?,?)', (title,category))
    conn.commit()

In [6]:
connect()
create_table()

In [7]:
url_it = 'https://news.yahoo.co.jp/list/?c=computer&p='
url_sport = 'https://news.yahoo.co.jp/list/?c=sports&p='
url_economy = 'https://news.yahoo.co.jp/list/?c=economy&p='
url_science = 'https://news.yahoo.co.jp/list/?c=science&p='

for i in range(100):
    it_all_link = url_it+str(i + 1)
#     print(it_all_link)
    it_r = requests.get(it_all_link)
    it_url_bs = BeautifulSoup(it_r.text, 'html.parser')
    title_table = it_url_bs.find("ul", class_="list")
    for dt in title_table.find_all('dt'):
        title= dt.text
#         print(title)
        load(title,'IT') 


In [8]:
for i in range(100):
    sport_all_link = url_sport+str(i + 1)
#     print(it_all_link)
    sport_r = requests.get(sport_all_link)
    sport_url_bs = BeautifulSoup(sport_r.text, 'html.parser')
    title_table2 = it_url_bs.find("ul", class_="list")
    for dt in title_table2.find_all('dt'):
        title2= dt.text
#         print(title)
        load(title2,'SPORTS')  

In [9]:
for i in range(100):
    economy_all_link = url_economy+str(i + 1)
#     print(it_all_link)
    economy_r = requests.get(economy_all_link)
    economy_url_bs = BeautifulSoup(economy_r.text, 'html.parser')
    title_table3 = it_url_bs.find("ul", class_="list")
    for dt in title_table3.find_all('dt'):
        title3= dt.text
#         print(title)
        load(title3,'ECONOMY') 

In [10]:
for i in range(100):
    science_all_link = url_science+str(i + 1)
#     print(it_all_link)
    science_r = requests.get(science_all_link)
    science_url_bs = BeautifulSoup(science_r.text, 'html.parser')
    title_table4 = it_url_bs.find("ul", class_="list")
    for dt in title_table4.find_all('dt'):
        title4= dt.text
#         print(title)
        load(title4,'SCIENCE')  

In [11]:
close()

In [5]:
con = None
con = sqlite3.connect("./yahoo_1.db")
cursor = con.cursor()
res = cursor.execute("SELECT title FROM news")
bs = cursor.fetchall()
print(bs)

[('無限アラート 2人不起訴処分',), ('TDS 噴水に足入れて涼むのNG',), ('ポケモン 睡眠をエンタメ化',), ('ポケモン 新作ゲーム2本発表',), ('ポケモン DeNAと新作ゲーム',), ('海外で共感? スマホNG結婚式',), ('新iPod touch ゲームに重点',), ('新型iPod touch 突如発表',), ('スマホ商戦に異状 上位に逆風',), ('ビットコイン価格回復 背景は',), ('見城氏 日本国紀への批判一蹴',), ('林死刑囚長男 発信する理由',), ('賞贈呈式 作家が幻冬舎を批判',), ('かぶる傘 コラ画像のデマ拡散',), ('大統領と握手の作家 招待否定',), ('外資規制にITも 情報流出防ぐ',), ('配膳ロボット導入 現場の声は',), ('ハムスター動画で「虐待」',), ('米の華為排除 もろ刃の剣',), ('商談の良しあし ソフトが判定',), ('不謹慎狩り 続く不毛な連鎖',), ('華為スマホ出荷 最大24%減',), ('「パオチャン」が活動休止へ',), ('SNS炎上「防災訓練」が人気',), ('安全ピンで痴漢撃退過剰?賛否',), ('在宅で稼げる トラブルに注意',), ('荒野行動とPUBG 和解が成立',), ('NGT支配人Twitter開始に賛否',), ('FB偽アカ 半年で30億件超削除',), ('ネット弱い主婦 加害者にも',), ('政府のドローン 8割が中国製',), ('スパコン 京の後継は「富岳」',), ('AIが服コーデ提案 ゾゾ子会社',), ('社長のツイート 幻冬舎が謝罪',), ('華為の対応疑問 顧客離れ危機',), ('焼きそばパン 愛される秘密は',), ('ZOZO 実店舗も「ツケ払い」',), ('QRコードで扉制御へ 浅草線',), ('華為スマホ ドコモも予約停止',), ('SBとKDDI 華為スマホ発売延期',), ('偽医師がデマ発信 信頼どこに',), ('はあちゅう氏 アンチ無視せず',), ('満員電車 理想の動き方を分析',), ('ファーウェイ 独自OS投入か',), ('ゲーム依存 医師語る「怖さ」',), ('河野外相が謎ツイート 話題',), ('LINE Payの還元 1日で90

In [6]:
def mecab_analysis(texts):
    t = mc.Tagger("")
    t.parse('')
    output = []
    node =  t.parseToNode(texts)  
    while node:
        if node.surface != "":  # ヘッダとフッタを除外
            word_type = node.feature.split(",")[0]
            if word_type in ['名詞']:
                output.append(node.surface)
        node = node.next
        if node is None:
            break
    return output

In [8]:
magachi = [mecab_analysis(text[0]) for text in bs]
magachi
# word_text = [w[0] for w in bs]

[['無限', 'アラート', '2', '人', '起訴', '処分'],
 ['TDS', '噴水', '足', 'の', 'NG'],
 ['ポケモン', '睡眠', 'エン', 'タメ', '化'],
 ['ポケモン', '新作', 'ゲーム', '2', '本', '発表'],
 ['ポケモン', 'DeNA', '新作', 'ゲーム'],
 ['海外', '共感', '?', 'スマホ', 'NG', '結婚式'],
 ['iPod', 'touch', 'ゲーム', '重点'],
 ['新型', 'iPod', 'touch', '発表'],
 ['スマホ', '商戦', '異状', '上位', '逆風'],
 ['ビット', 'コイン', '価格', '回復', '背景'],
 ['見城', '氏', '日本国', '紀', '批判', '一蹴'],
 ['林', '死刑', '囚', '長男', '発信', '理由'],
 ['賞', '贈呈', '式', '作家', '幻冬舎', '批判'],
 ['傘', 'コラ', '画像', 'デマ', '拡散'],
 ['大統領', '握手', '作家', '招待', '否定'],
 ['外資', '規制', 'IT', '情報', '流出'],
 ['配膳', 'ロボット', '導入', '現場', '声'],
 ['ハムスター', '動画', '虐待'],
 ['米', '華', '為', '排除', 'もろ刃', '剣'],
 ['商談', '良しあし', 'ソフト', '判定'],
 ['不謹慎', '不毛', '連鎖'],
 ['華', '為', 'スマホ', '出荷', '最大', '24', '%', '減'],
 ['パオ', 'チャン', '活動', '休止'],
 ['SNS', '炎上', '防災', '訓練', '人気'],
 ['安全ピン', '痴漢', '撃退', '過剰', '?', '賛否'],
 ['在宅', 'トラブル', '注意'],
 ['荒野', '行動', 'PUBG', '和解', '成立'],
 ['NGT', '支配人', 'Twitter', '開始', '賛否'],
 ['FB', 'アカ', '半年', '30', '億', '件', '削除'],


In [9]:
dictionary = corpora.Dictionary(magachi)
print(dictionary.token2id)

{'2': 0, 'アラート': 1, '人': 2, '処分': 3, '無限': 4, '起訴': 5, 'NG': 6, 'TDS': 7, 'の': 8, '噴水': 9, '足': 10, 'エン': 11, 'タメ': 12, 'ポケモン': 13, '化': 14, '睡眠': 15, 'ゲーム': 16, '新作': 17, '本': 18, '発表': 19, 'DeNA': 20, '?': 21, 'スマホ': 22, '共感': 23, '海外': 24, '結婚式': 25, 'iPod': 26, 'touch': 27, '重点': 28, '新型': 29, '上位': 30, '商戦': 31, '異状': 32, '逆風': 33, 'コイン': 34, 'ビット': 35, '価格': 36, '回復': 37, '背景': 38, '一蹴': 39, '批判': 40, '日本国': 41, '氏': 42, '紀': 43, '見城': 44, '囚': 45, '林': 46, '死刑': 47, '理由': 48, '発信': 49, '長男': 50, '作家': 51, '幻冬舎': 52, '式': 53, '賞': 54, '贈呈': 55, 'コラ': 56, 'デマ': 57, '傘': 58, '拡散': 59, '画像': 60, '否定': 61, '大統領': 62, '招待': 63, '握手': 64, 'IT': 65, '外資': 66, '情報': 67, '流出': 68, '規制': 69, 'ロボット': 70, '声': 71, '導入': 72, '現場': 73, '配膳': 74, 'ハムスター': 75, '動画': 76, '虐待': 77, 'もろ刃': 78, '剣': 79, '排除': 80, '為': 81, '米': 82, '華': 83, 'ソフト': 84, '判定': 85, '商談': 86, '良しあし': 87, '不毛': 88, '不謹慎': 89, '連鎖': 90, '%': 91, '24': 92, '出荷': 93, '最大': 94, '減': 95, 'チャン': 96, 'パオ': 97, '休止': 98, '活動': 99,

In [20]:
# vec_all = []
# for i in range(8000):
#     vec = dictionary.doc2bow(magachi[i])
#     vec_all.append(list(matutils.corpus2dense([vec], num_terms=len(dictionary)).T[0]))

In [21]:
# dense_list = [list(matutils.corpus2dense(vec_all, num_terms=len(dictionary)).T[0])]

TypeError: 'numpy.float32' object is not iterable

In [10]:
dense_list = []
for vc in magachi:
    vec = dictionary.doc2bow(vc)
    dense = list(matutils.corpus2dense([vec],num_terms=len(dictionary)).T[0])
    dense_list.append(dense)

In [11]:
X = dense_list
Y = []
for i in range(8000):
    if i<2000:
        Y.append('0')
    elif i<4000:
        Y.append('1')
    elif i<6000:
        Y.append('2')
    elif i<8000:
        Y.append('3')
            
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)

In [12]:
random_forest= ensemble.RandomForestClassifier(max_depth=20)
random_forest.fit(X_train, Y_train)
random_forest.score(X_test, Y_test)

/home/r-ei/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.416875

In [13]:
param_grid = {
             "n_estimators": [50,100,200,300,400,500],
             "max_features": [1,3,10],
             "min_samples_split": [2.,3,10],
             "min_samples_leaf": [1,3,10],
             "bootstrap": [True, False],
             "criterion": ["gini", "entropy"]}

In [ ]:
forest_grid = GridSearchCV(estimator=ensemble.RandomForestClassifier(random_state=0),
                          param_grid=param_grid,
                          scoring="accuracy",
                          cv = 3,
                          n_jobs = 5)
forest_grid.fit(X_train,Y_train)

In [ ]:
print(forest_grid.best_params)